In [5]:
import requests
from bs4 import BeautifulSoup
import re

import firebase_admin
from firebase_admin import credentials, firestore

from recipe_scrapers import scrape_me
import requests
from recipe_scrapers import scrape_html


In [3]:

path = 'supermercado-371012-firebase-adminsdk-lu3fc-48a8c3583d.json'
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate(path)
    firebase_admin.initialize_app(cred)

firebase_admin.get_app()

db = firestore.client()

In [4]:
def crawl_recipe_links(page):
    # Send the request and get the response
    response = requests.get(page)

    # Parse the response content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements with the class 'c-teaser__link'
    recipe_elements = soup.find_all(class_='ds-recipe-card__image-wrap ds-teaser-link__image-wrap ds-teaser-link__image-wrap')
    print(len(recipe_elements))

    # Print the text and href attributes of each element
    recipe_links = []
    for html in recipe_elements:
        html = str(html)
        soup = BeautifulSoup(html, 'html.parser')
        
        # find the img tag
        img_tag = soup.find('img')

        # access the src attribute
        src = img_tag['src']
        substring_index = src.index('bilder')
        cutoff_string = src[:substring_index]
        recipe_links.append(cutoff_string)
    return recipe_links


In [58]:
def write_recipe(title, time, ingredients, instructions, image, nutrients):
    doc_ref = db.collection(u'recipes_veggie').document(title)
    doc_ref.set({

        u'title': title,
        u'time': time,
        u'ingredients': ingredients,
        u'instructions': instructions,
        u'image': image,
        u'nutrients': nutrients,

})

In [56]:
def get_recipe_ingredients(url):
    html = requests.get(url).content

    scraper = scrape_html(html=html, org_url=url)

    write_recipe(scraper.title(),scraper.total_time(), scraper.ingredients(), scraper.instructions(), scraper.image(),scraper.nutrients())
    print(scraper.title())


get_recipe_ingredients("https://www.chefkoch.de/rezepte/2737891426025016/Gyros-Nudelauflauf-in-Metaxasosse.html")

Gyros-Nudelauflauf in Metaxasoße


In [59]:
recipe_links = crawl_recipe_links('https://www.chefkoch.de/rs/s0t21,32/Hauptspeise-Vegetarisch-Rezepte.html')

for i, url in enumerate(recipe_links):
    get_recipe_ingredients(url)

59
Kartoffel-Brokkoli-Auflauf
Bester vegetarischer Nudelauflauf
Vegetarische Frikadellen
Würziges Kartoffelcurry
Paprika-Reispfanne mit Joghurtsauce
Vegetarische Spinat-Gemüse-Lasagne mit Tomatensoße
Vegetarische Tortellini-Pfanne
Parmesan-Knoblauch-Kartoffelecken
Schnelles Spinat-Kichererbsen-Gericht
Gebratene Nudeln mit Gemüse, asiatisch
Schwarzwälder Pilz-Dim Sum mit Sojasauce
Weißer Ofenspargel mit Graupen-Wildkräutersalat
Grießbrei von Großmutter
Käsespätzle
Maiscremesuppe mit Bohnen-Topping
Penne mit Pilz-Ragout und Ei
Steinpilzrisotto mit Topinambur und gelben Karotten
Tortellini in Gorgonzola-Sauce
Spaghetti aglio, olio e peperoncino
Risotto alla milanese
Spargel aus dem Ofen - ideal für Gäste
Bulgur Buddha Bowl
Gebackene Süßkartoffeln und Rote Bete mit Kräutersalat und Tahin-Dip
Rote Linsen-Curry mit Süßkartoffeln
Italienischer Pizzateig
Cremiger Tortellini-Topf
Gratinierter Brokkoli mit Schafskäse
Pizzateig
Pasta mit Kürbis-Pesto
Rote Linsen-Kokos-Suppe
Julies feine Gemüselas